In [1]:
using BenchmarkTools
using NPZ
using JSON

# Common

In [2]:
struct Point
    x::Float64
    y::Float64
end

# Direction vector
struct DVector
    x::Float64
    y::Float64
end

function cross_product(u::DVector, v::DVector)
    u.x * v.y - u.y * v.x
end

function dot_product(u::DVector, v::DVector)
    u.x * v.x + u.y * v.y
end

dot_product (generic function with 1 method)

# Sutherland-Hodgman

In [3]:
function _inside(p::Point, r::Point, U::DVector)
    U.x * (p.y - r.y) > U.y * (p.x - r.x)
end

function _intersection(a::Point, V::DVector, r::Point, N::DVector)
    W = DVector(r.x - a.x, r.y - a.y)
    nw = dot_product(N, W)
    nv = dot_product(N, V)
    if nv != 0.0
        t = nw / nv
        return true, Point(a.x + t * V.x, a.y + t * V.y)
    else
        return false, Point(0.0, 0.0)
    end
end

function polygon_area(polygon, len)
    area = 0.0
    a = Point(polygon[1, 1], polygon[2, 1])
    b = Point(polygon[1, 2], polygon[2, 2])
    U = DVector(b.x - a.x, b.y - a.y)
    for i in 3:len
        c = Point(polygon[1, i], polygon[2, i])
        V = DVector(a.x - c.x, a.y - c.y)
        area += abs(cross_product(U, V))
        b = c
        U = V
    end
    return 0.5 * area
end

function _copy(src, dst, n)
    for i in 1:n
        dst[1, i] = src[1, i]
        dst[2, i] = src[2, i]
    end
end

function _push(array, n, value)
    array[1, n + 1] = value.x
    array[2, n + 1] = value.y
    return n + 1
end

function clip_polygons(polygon, clipper)
    n_output = size(polygon)[2]
    n_clip = size(clipper)[2]
    output = Array{Float64}(undef, (2, 6))
    subject = Array{Float64}(undef, (2, 6))
    _copy(polygon, output, n_output)
    
    r = Point(clipper[1, n_clip], clipper[2, n_clip])
    for i in 1:n_clip
        s = Point(clipper[1, i], clipper[2, i])

        U = DVector(s.x - r.x, s.y - r.y)
        N = DVector(-U.y, U.x)
        if U.x == 0.0 && U.y == 0.0
            continue
        end
        len = n_output
        _copy(output, subject, len)
        n_output = 0

        a = Point(subject[1, len], subject[2, len])
        a_inside = _inside(a, r, U)
        for j in 1:len
            b = Point(subject[1, j], subject[2, j])

            V = DVector(b.x - a.x, b.y - a.y)
            if V.x == 0.0 && V.y == 0.0
                continue
            end
            b_inside = _inside(b, r, U)
            
            if b_inside
                if !a_inside
                    succes, point = _intersection(a, V, r, N)
                    if succes
                        n_output = _push(output, n_output, point)
                    end
                end
                n_output = _push(output, n_output, b)
            elseif a_inside
                succes, point = _intersection(a, V, r, N)
                if succes
                    n_output = _push(output, n_output, point)
                else
                    b_inside = true
                    n_output = _push(output, n_output, b)
                end
            end
            
            a = b
            a_inside = b_inside
        end
        
        if n_output < 3
            return 0.0
        end
        
        r = s
    end
    area = polygon_area(output, n_output)
    return area
end      

clip_polygons (generic function with 1 method)

# Read triangles

In [4]:
a = permutedims(npzread("triangles_a.npy"), (3, 2, 1))
b = permutedims(npzread("triangles_b.npy"), (3, 2, 1))

2×3×10000000 Array{Float64,3}:
[:, :, 1] =
 0.772866  0.570218  0.250386
 0.609051  0.958381  0.258065

[:, :, 2] =
 0.264513  0.664046  0.124763
 0.538956  0.928762  0.803564

[:, :, 3] =
 0.47856   0.674365  0.393399
 0.301384  0.281799  0.901712

...

[:, :, 9999998] =
 0.398022  0.0983778  0.43207
 0.561544  0.233543   0.38628

[:, :, 9999999] =
 0.982461  0.107067  0.588445
 0.401328  0.310718  0.162313

[:, :, 10000000] =
 0.529699  0.942806  0.724126
 0.150737  0.965228  0.731719

In [5]:
function area_of_intersection(a, b)
    n = size(a)[end]
    out = Array{Float64}(undef, n)
    for i in 1:n
        t0 = a[:, :, i]
        t1 = b[:, :, i]
        out[i] = clip_polygons(t0, t1)
    end
    return out
end

area_of_intersection (generic function with 1 method)

In [6]:
times = Dict()
for s in [1, 10, 100, 1_000, 10_000, 100_000, 1_000_000, 10_000_000]
    aa = a[:, :, 1:s]
    bb = b[:, :, 1:s]
    res = @belapsed(area_of_intersection($aa, $bb))
    times[s] = res
end

open("timing/julia.json","w") do f
    JSON.print(f, times)
end

npzwrite("answers/julia.npy", area_of_intersection(a[:, :, 1:10_000], b[:, :, 1:10_000]))